# SciStream Fabric Example UTAH ---- SALT 
### This exmaple test only the SciStream Protocol itself. According to the basic setups, this version may not need switches.
This notebook will setup a demo testbed for Scistream by utilizing Fabric resources. Will support QUIC implementation between channels. And will evaluate the performance through various streaming protocols (e.g., DASH).

Additional resources:
- [SciStream](https://scistream.github.io)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [SciStream Repo](https://github.com/scistream)
- [SciStream YouTube Presentations](https://youtu.be/IWKGjAa_TOA)
- [Topology Graph](https://drive.google.com/file/d/124xh4K15c7tUcaETXmj-JSPKcQLA_oTC/view?usp=sharing)

In [1]:
 import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Set your FABRIC PROJECT ID
os.environ['FABRIC_PROJECT_ID']='f4344c37-52e3-4d0c-a354-9311342ec109'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='cqy78_0038438951'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/fabric_bastion_key' #'id_rsa_fabric'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/work/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/work/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Basic FABRIC Slice Configuration

In [2]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [3]:
try:
    available_resources = fablib.get_available_resources()
    print(f"Available Resources: {available_resources}")
except Exception as e:
    print(f"Error: {e}")

Available Resources: Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
UCSD        10  320/320  2560/2560  116400/116400  635/635                 2/2                             4/4                            16/16               4/4               6/6
SALT         6  128/192  1280/1536  60580/60600    377/381                 2/2                             2/2                            10/10               2/2               3/3
UTAH        10  236/320  2246/2560  116258/116400  619/635                 2/2                             4/4                            16/16               4/4               5/5
MAX         10  252/320  2350/2560  115994/116400  578/63

## Configure Slice Parameters

This section builds the experiment slice. TO BE CHANGED, according to the code. this setup may change.

Note: Fabric did not support modifying resources after create slides. 



In [3]:
# Slice 
slice_name = 'SciStream-UTAH-SALT'

# Switches --- This version may not need any switch
# s_name = "s1" 

# switch_cores = 2
# switch_ram = 8
# switch_disk = 100

# Hosts
h1_name = "h1" # consider as Facility 1 ProdApp
h2_name = "h2" # consider as Facility 1 ProdGN
h3_name = "h3" # consider as S2UC
h4_name = "h4" # consider as Facility 2 ConsGN
h5_name = "h5" # consider as Facility 2 ConsApp

h1_ip="192.168.0.1"
h2_ip="192.168.0.2" # prodGN - command: listener-ip: 192.168.0.2 s2-port 5000
h3_ip="192.168.0.3" # S2UC json file running prod and cons on 5000 but with different ip address
h4_ip="192.168.0.4" # consGN - command: listener-ip: 192.168.0.4 s2-port 5000
h5_ip="192.168.0.5"

h1_LAN_ip="172.16.0.1"
h2_LAN_ip="172.16.0.2"

h2_WAN_ip="172.16.1.1"
h4_WAN_ip="172.16.1.2"

h4_LAN_ip="172.16.2.1"
h5_LAN_ip="172.16.2.2"

host_cores = 32
host_ram = 128
host_disk = 10
# Sites
# Default set to TACC to support IPv4 and github clone...
# Should use proxy to access github once checked as IPv6 address. Or use other git platfrom, e.g., bibucket...
site_1 = 'UTAH' # 'SALT' # 'NCSA' # s'WASH' # 'MAX' # 'STAR' # 'NCSA' # 'MAX' # 'UTAH' # 'DALL' # 'TACC' # 'MAX'
site_2 = 'SALT' # 'SALT' # 'NCSA' # s'WASH' # 'MAX' # 'STAR' # 'NCSA' # 'MAX' # 'UTAH' # 'DALL' # 'TACC' # 'MAX'

control_net_name = 'control_net' #no need for S2DS test only
prod_lan_net_name = 'prod_lan_net'
gn_net_name = 'GN_WAN_NET'
cons_lan_net_name = 'cons_lan_net'

net_pub_name = 'open_net'

# All node properties
#username = 'ubuntu'
image = 'default_ubuntu_20'
vlan = '1000'
#image_type = 'qcow2'




### Create the Slice

**This part is optional. Once the slice on, there is no need to have this block running.** \
In terms of the NIC usage. We prefer NIC_ConnectX_6 or NIC_Basic. They both provide 100 Gbps. However, NIC_ConnectX_5 still accepable. \
The network topology will be fixed and non-programmable after creating the slice. Be care of setup l2 or l3 networks. \
Network can be setup cross-site. A vlan is established based on your settings. The RTT is based on the physical distances. \
*In the newest FABRIC release, project manager may need to permissions from Admin to add cross-cite communication. Please attention.*

In [5]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Add switch node
#     switch1 = slice.add_node(name=s_name, site=site_2)
#     switch1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
#     switch1.set_image(image)
#     #[s1_iface_local] = s1.add_component(model='NIC_Basic', name="s1_local_nic").get_interfaces()
#     [s_iface_to_h1, s_iface_to_h2] = switch1.add_component(model='NIC_ConnectX_6', name="prod_switch_nic").get_interfaces()
#     s_iface_to_h1.set_vlan(vlan=vlan)
#     s_iface_to_h2.set_vlan(vlan=vlan)

#     s_iface_to_h3 = switch1.add_component(model='NIC_Basic', name="uc_local_nic").get_interfaces()[0]
#     s_iface_to_h3.set_vlan(vlan=vlan)
#     [s_iface_to_h4, s_iface_to_h5] = switch1.add_component(model='NIC_ConnectX_6', name="cons_switch_nic").get_interfaces()
#     s_iface_to_h4.set_vlan(vlan=vlan)
#     s_iface_to_h5.set_vlan(vlan=vlan)
    
    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site_1)
    h1.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h1.set_image(image)
    h1_lan_iface = h1.add_component(model='NIC_Basic', name="h1_nic").get_interfaces()[0]
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site_1)
    h2.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h2.set_image(image)
    h2_lan_iface = h2.add_component(model='NIC_Basic', name="h2_nic").get_interfaces()[0]
    wan_iface1 = h2.add_component(model='NIC_Basic', name="h2_wan_nic").get_interfaces()[0]
 
    # Add host node h3 (no need for S2DS only test, but required for whole SciStream test)
    # h3 = slice.add_node(name=h3_name, site=site_2)
    # h3.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    # h3.set_image(image)
    # h3_iface = h3.add_component(model='NIC_Basic', name="h3_nic").get_interfaces()[0]
    
    # Add host node h4
    h4 = slice.add_node(name=h4_name, site=site_2)
    h4.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h4.set_image(image)
    h4_lan_iface = h4.add_component(model='NIC_Basic', name="h4_nic").get_interfaces()[0]
    wan_iface2 = h4.add_component(model='NIC_Basic', name="h4_wan_nic").get_interfaces()[0]
    
    # Add host node h5
    h5 = slice.add_node(name=h5_name, site=site_2)
    h5.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h5.set_image(image)
    h5_lan_iface = h5.add_component(model='NIC_Basic', name="h5_nic").get_interfaces()[0]
    

    #Add control panel networks 
    # control_net = slice.add_l2network(name=control_net_name, interfaces=[h1_iface,h2_iface, h3_iface, h4_iface,h5_iface])
    
    #Add Prod LAN
    prod_lan = slice.add_l2network(name=prod_lan_net_name, interfaces=[h1_lan_iface,h2_lan_iface])
    #Add Cons LAN
    cons_lan = slice.add_l2network(name=cons_lan_net_name, interfaces=[h4_lan_iface,h5_lan_iface])
    
    #Add GN WAN
    gn_wan = slice.add_l2network(name=gn_net_name, interfaces=[wan_iface1,wan_iface2])

    
    #Add L3 network for S2UC (May get error from Fabric)
    #net2 = slice.add_l3network(name=net_pub_name, interfaces=[h3_iface_pub], type='IPv4')
    
    #Submit Slice Request
    slice.submit() #(wait_progress=True)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    


---------------  ------------------------------------
Slice Name       SciStream-UTAH-SALT
Slice ID         8e9e5c21-7ccb-4ac0-9c18-bdcc54cfb31a
Slice State      StableOK
Lease End (UTC)  2022-08-19 03:48:46 +0000
---------------  ------------------------------------

Retry: 15, Time: 186 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image              Management IP                           State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  -----------------  --------------------------------------  -------  -------
34a2be8e-6ac9-4f93-a7c6-9a20ff803dca  h1      UTAH    utah-w5.fabric-testbed.net       32    128      10  default_ubuntu_20  2001:1948:417:7:f816:3eff:fe34:634a     Active
561c677f-2fc2-4c02-96bb-b0e64029416f  h2      UTAH    utah-w5.fabric-testbed.net       32    128      10  default_ubuntu_20  2001:1948:417:7:f816:3eff:fef3:a359     Activ

# timeout needs to be extended

## Get the Slice

In [4]:
# ssh -F ~/work/fabric_config/ssh_config -i ~/work/id_rsa 
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"Node: {node.get_ssh_command()}")
except Exception as e:
    print(f"Exception: {e}")

Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2001:1948:417:7:f816:3eff:fe34:634a
Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2001:1948:417:7:f816:3eff:fef3:a359
Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2001:400:a100:3010:f816:3eff:fe9b:922
Node: ssh -i /home/fabric/work/id_rsa -J cqy78_0038438951@bastion-1.fabric-testbed.net ubuntu@2001:400:a100:3010:f816:3eff:fedb:bf2e


## Configure Nodes
 ### This is just an exmple. Please wait for github to turn public and then run the script. Or the system will wait for cerdential.
 ### STEP 1: Setup controll net ---NO NEED

 ### STEP 2: Setup LAN net

In [7]:
try:
    h1 = slice.get_node(name=h1_name)        
    h1_os_iface = h1.get_interface(network_name=prod_lan_net_name)
    h1_os_iface.set_ip(ip=h1_LAN_ip, cidr="24")
    
    print(f"ifname: {h1_os_iface.get_os_interface()}")
    print(f"mac: {h1_os_iface.get_mac()}")
    stdout, stderr = h1.execute(f'sudo ip addr list {h1_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h1.execute(host_config_script_h1)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h2 = slice.get_node(name=h2_name)
    h2_os_iface = h2.get_interface(network_name=prod_lan_net_name)
    h2_os_iface.set_ip(ip=h2_LAN_ip, cidr="24")
 
    print(f"ifname: {h2_os_iface.get_os_interface()}")
    print(f"mac: {h2_os_iface.get_mac()}")
    stdout, stderr = h2.execute(f'sudo ip addr list {h2_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h2.execute(host_config_script_h2)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    

try:
    h4 = slice.get_node(name=h4_name)
    h4_os_iface = h4.get_interface(network_name=cons_lan_net_name)
    h4_os_iface.set_ip(ip=h4_LAN_ip, cidr="24")

    print(f"ifname: {h4_os_iface.get_os_interface()}")
    print(f"mac: {h4_os_iface.get_mac()}")
    stdout, stderr = h4.execute(f'sudo ip addr list {h4_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h5 = slice.get_node(name=h5_name)
    h5_os_iface = h5.get_interface(network_name=cons_lan_net_name)
    h5_os_iface.set_ip(ip=h5_LAN_ip, cidr="24")

    print(f"ifname: {h5_os_iface.get_os_interface()}")
    print(f"mac: {h5_os_iface.get_mac()}")
    stdout, stderr = h5.execute(f'sudo ip addr list {h5_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

ifname: ens7
mac: 02:13:80:A3:DC:9D
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:13:80:a3:dc:9d brd ff:ff:ff:ff:ff:ff
    inet 172.16.0.1/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::13:80ff:fea3:dc9d/64 scope link 
       valid_lft forever preferred_lft forever


ifname: ens7
mac: 02:29:C7:8C:E1:88
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:29:c7:8c:e1:88 brd ff:ff:ff:ff:ff:ff
    inet 172.16.0.2/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::29:c7ff:fe8c:e188/64 scope link 
       valid_lft forever preferred_lft forever


ifname: ens8
mac: 02:25:EF:C4:AE:D0
4: ens8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:25:ef:c4:ae:d0 brd ff:ff:ff:ff:ff:ff
    inet 172.16.2.1/24 scope global ens8
       valid_lft forever preferr

 ### STEP 3: Setup WAN net

In [8]:
try:
    h2 = slice.get_node(name=h2_name)
    h2_os_iface = h2.get_interface(network_name=gn_net_name)
    h2_os_iface.set_ip(ip=h2_WAN_ip, cidr="24")
 
    print(f"ifname: {h2_os_iface.get_os_interface()}")
    print(f"mac: {h2_os_iface.get_mac()}")
    stdout, stderr = h2.execute(f'sudo ip addr list {h2_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h2.execute(host_config_script_h2)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    

try:
    h4 = slice.get_node(name=h4_name)
    h4_os_iface = h4.get_interface(network_name=gn_net_name)
    h4_os_iface.set_ip(ip=h4_WAN_ip, cidr="24")

    print(f"ifname: {h4_os_iface.get_os_interface()}")
    print(f"mac: {h4_os_iface.get_mac()}")
    stdout, stderr = h4.execute(f'sudo ip addr list {h4_os_iface.get_os_interface()}')
    print (stdout)
    print (stderr)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

ifname: ens8
mac: 02:AA:1F:20:41:A6
4: ens8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:aa:1f:20:41:a6 brd ff:ff:ff:ff:ff:ff
    inet 172.16.1.1/24 scope global ens8
       valid_lft forever preferred_lft forever
    inet6 fe80::aa:1fff:fe20:41a6/64 scope link 
       valid_lft forever preferred_lft forever


ifname: ens7
mac: 02:09:16:54:61:EB
3: ens7: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:09:16:54:61:eb brd ff:ff:ff:ff:ff:ff
    inet 172.16.1.2/24 scope global ens7
       valid_lft forever preferred_lft forever
    inet6 fe80::9:16ff:fe54:61eb/64 scope link 
       valid_lft forever preferred_lft forever




## TCP Tunning and iperf test

In [9]:
stdout, stderr = h1.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')
stdout, stderr = h2.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')

stdout, stderr = h4.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')
stdout, stderr = h5.execute(f'sudo apt update && sudo apt install -y iperf iperf3 net-tools')

In [10]:
stdout, stderr = h1.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')

stdout, stderr = h2.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')


stdout, stderr = h4.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')

stdout, stderr = h5.execute('echo "net.core.rmem_max = 536870912\nnet.core.wmem_max = 536870912\nnet.ipv4.tcp_rmem = 4096 87380 536870912\nnet.ipv4.tcp_wmem = 4096 65536 536870912\nnet.ipv4.tcp_congestion_control=htcp\nnet.ipv4.tcp_mtu_probing=1\nnet.core.default_qdisc = fq\n" | sudo tee -a /etc/sysctl.conf && sudo sysctl -p')


In [11]:
#ATTENTION: be aware of the NIC id!
#NEED SUDO! 
stdout, stderr = h1.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')
print (stdout)
print (stderr)
stdout, stderr = h2.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')
stdout, stderr = h2.execute('sudo ifconfig ens8 mtu 8900 up && sudo tc qdisc add dev ens8 root fq maxrate 30gbit')
print (stdout)
print (stderr)


stdout, stderr = h4.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')
stdout, stderr = h4.execute('sudo ifconfig ens8 mtu 8900 up && sudo tc qdisc add dev ens8 root fq maxrate 30gbit')
print (stdout)
print (stderr)
stdout, stderr = h5.execute('sudo ifconfig ens7 mtu 8900 up && sudo tc qdisc add dev ens7 root fq maxrate 30gbit')
print (stdout)
print (stderr)

In [12]:

h2 = slice.get_node(name=h2_name) 
stdout, stderr = h2.execute('tmux new -d \'iperf -s -f K\'')
print (stdout)
print (stderr)

In [13]:
#TCP TEST:
#Forward direction:
h1 = slice.get_node(name=h1_name) 
stdout, stderr = h1.execute(f'iperf -c 172.16.0.2 -P 32 -w 999M -t 30') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.0.2, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 35] local 172.16.0.1 port 56380 connected with 172.16.0.2 port 5001
[ 29] local 172.16.0.1 port 56370 connected with 172.16.0.2 port 5001
[ 32] local 172.16.0.1 port 56378 connected with 172.16.0.2 port 5001
[ 33] local 172.16.0.1 port 56376 connected with 172.16.0.2 port 5001
[ 30] local 172.16.0.1 port 56372 connected with 172.16.0.2 port 5001
[  4] local 172.16.0.1 port 56318 connected with 172.16.0.2 port 5001
[ 31] local 172.16.0.1 port 56374 connected with 172.16.0.2 port 5001
[  7] local 172.16.0.1 port 56324 connected with 172.16.0.2 port 5001
[  6] local 172.16.0.1 port 56326 connected with 172.16.0.2 port 5001
[  5] local 172.16.0.1 port 56322 connected with 172.16.0.2 port 5001
[ 11] local 172.16.0.1 port 56330 connected with 172.16.0.2 port 5001
[

In [14]:
#Backward direction:
h4 = slice.get_node(name=h4_name) 
stdout, stderr = h4.execute(f'iperf -c 172.16.1.1 -P 32 -w 999M -t 30') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.1.1, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 34] local 172.16.1.2 port 33424 connected with 172.16.1.1 port 5001
[ 31] local 172.16.1.2 port 33422 connected with 172.16.1.1 port 5001
[ 32] local 172.16.1.2 port 33418 connected with 172.16.1.1 port 5001
[ 33] local 172.16.1.2 port 33420 connected with 172.16.1.1 port 5001
[  3] local 172.16.1.2 port 33366 connected with 172.16.1.1 port 5001
[  4] local 172.16.1.2 port 33362 connected with 172.16.1.1 port 5001
[ 10] local 172.16.1.2 port 33376 connected with 172.16.1.1 port 5001
[  8] local 172.16.1.2 port 33370 connected with 172.16.1.1 port 5001
[  5] local 172.16.1.2 port 33364 connected with 172.16.1.1 port 5001
[  7] local 172.16.1.2 port 33372 connected with 172.16.1.1 port 5001
[  9] local 172.16.1.2 port 33374 connected with 172.16.1.1 port 5001
[

In [15]:
#UDP TEST:
#Forward direction
h1 = slice.get_node(name=h1_name) 
stdout, stderr = h1.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.0.2 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.0.2, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  4] local 172.16.0.1 port 51710 connected with 172.16.0.2 port 5001
[  5] local 172.16.0.1 port 34999 connected with 172.16.0.2 port 5001
[  3] local 172.16.0.1 port 44047 connected with 172.16.0.2 port 5001
[  6] local 172.16.0.1 port 50514 connected with 172.16.0.2 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0-10.0 sec  6.31 GBytes  5.42 Gbits/sec
[  3] Sent 765215 datagrams
[  4]  0.0-10.0 sec  10.4 GBytes  8.96 Gbits/sec
[  4] Sent 1266286 datagrams
[  5]  0.0-10.0 sec  10.1 GBytes  8.70 Gbits/sec
[  5] Sent 1229160 datagrams
[  6]  0.0-10.0 sec  10.2 GBytes  8.79 Gbits/sec
[  6] Sent 1241439 datagrams
[SUM]  0.0-10.0 sec  37.1 GBytes  31.9 Gbits/sec
[SUM] Sent 4502100 dat

In [16]:
#Backward direction
h4 = slice.get_node(name=h4_name) 
stdout, stderr = h4.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.1.1 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.1.1, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  6] local 172.16.1.2 port 40769 connected with 172.16.1.1 port 5001
[  3] local 172.16.1.2 port 37407 connected with 172.16.1.1 port 5001
[  4] local 172.16.1.2 port 57179 connected with 172.16.1.1 port 5001
[  5] local 172.16.1.2 port 51528 connected with 172.16.1.1 port 5001
[ ID] Interval       Transfer     Bandwidth
[  5]  0.0-10.0 sec  8.77 GBytes  7.54 Gbits/sec
[  5] Sent 1064727 datagrams
[  3]  0.0-10.0 sec  7.03 GBytes  6.04 Gbits/sec
[  3] Sent 853620 datagrams
[  6]  0.0-10.0 sec  4.40 GBytes  3.78 Gbits/sec
[  6] Sent 534294 datagrams
[  4]  0.0-10.0 sec  8.51 GBytes  7.31 Gbits/sec
[  4] Sent 1032385 datagrams
[SUM]  0.0-10.0 sec  28.7 GBytes  24.7 Gbits/sec
[SUM] Sent 3485026 data

In [17]:
h4 = slice.get_node(name=h4_name) 
stdout, stderr = h4.execute('tmux new -d \'iperf -s -f K\'')
print (stdout)
print (stderr)

In [18]:
#TCP TEST:
#Forward direction
h2 = slice.get_node(name=h2_name) 
stdout, stderr = h2.execute(f'iperf -c 172.16.1.2 -P 32 -w 999M') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.1.2, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 31] local 172.16.1.1 port 37394 connected with 172.16.1.2 port 5001
[ 29] local 172.16.1.1 port 37386 connected with 172.16.1.2 port 5001
[ 32] local 172.16.1.1 port 37388 connected with 172.16.1.2 port 5001
[ 33] local 172.16.1.1 port 37390 connected with 172.16.1.2 port 5001
[ 34] local 172.16.1.1 port 37392 connected with 172.16.1.2 port 5001
[  4] local 172.16.1.1 port 37332 connected with 172.16.1.2 port 5001
[  7] local 172.16.1.1 port 37338 connected with 172.16.1.2 port 5001
[  5] local 172.16.1.1 port 37334 connected with 172.16.1.2 port 5001
[  9] local 172.16.1.1 port 37340 connected with 172.16.1.2 port 5001
[ 11] local 172.16.1.1 port 37346 connected with 172.16.1.2 port 5001
[ 12] local 172.16.1.1 port 37348 connected with 172.16.1.2 port 5001
[

In [19]:
#Backward direction
h5 = slice.get_node(name=h5_name) 
stdout, stderr = h5.execute(f'iperf -c 172.16.2.1 -P 32 -w 999M') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.2.1, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 34] local 172.16.2.2 port 45428 connected with 172.16.2.1 port 5001
[ 33] local 172.16.2.2 port 45424 connected with 172.16.2.1 port 5001
[ 32] local 172.16.2.2 port 45422 connected with 172.16.2.1 port 5001
[ 31] local 172.16.2.2 port 45426 connected with 172.16.2.1 port 5001
[  7] local 172.16.2.2 port 45372 connected with 172.16.2.1 port 5001
[  3] local 172.16.2.2 port 45366 connected with 172.16.2.1 port 5001
[  5] local 172.16.2.2 port 45368 connected with 172.16.2.1 port 5001
[  4] local 172.16.2.2 port 45374 connected with 172.16.2.1 port 5001
[  6] local 172.16.2.2 port 45370 connected with 172.16.2.1 port 5001
[ 12] local 172.16.2.2 port 45376 connected with 172.16.2.1 port 5001
[  8] local 172.16.2.2 port 45378 connected with 172.16.2.1 port 5001
[

In [20]:
#UDP TEST:
#Forward direction
h2 = slice.get_node(name=h2_name) 
stdout, stderr = h2.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.1.2 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.1.2, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  5] local 172.16.1.1 port 47724 connected with 172.16.1.2 port 5001
[  3] local 172.16.1.1 port 32795 connected with 172.16.1.2 port 5001
[  4] local 172.16.1.1 port 40592 connected with 172.16.1.2 port 5001
[  6] local 172.16.1.1 port 42401 connected with 172.16.1.2 port 5001
[ ID] Interval       Transfer     Bandwidth
[  3]  0.0-10.0 sec  8.89 GBytes  7.64 Gbits/sec
[  3] Sent 1078907 datagrams
[  5]  0.0-10.0 sec  7.99 GBytes  6.86 Gbits/sec
[  5] Sent 969309 datagrams
[  4]  0.0-10.0 sec  4.03 GBytes  3.46 Gbits/sec
[  4] Sent 489296 datagrams
[  6]  0.0-10.0 sec  8.65 GBytes  7.43 Gbits/sec
[  6] Sent 1050264 datagrams
[SUM]  0.0-10.0 sec  29.6 GBytes  25.4 Gbits/sec
[SUM] Sent 3587776 data

In [21]:
#Backward direction
h5 = slice.get_node(name=h5_name) 
stdout, stderr = h5.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.2.1 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.2.1, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  6] local 172.16.2.2 port 33240 connected with 172.16.2.1 port 5001
[  3] local 172.16.2.2 port 44182 connected with 172.16.2.1 port 5001
[  4] local 172.16.2.2 port 34933 connected with 172.16.2.1 port 5001
[  5] local 172.16.2.2 port 50650 connected with 172.16.2.1 port 5001
[ ID] Interval       Transfer     Bandwidth
[  6]  0.0-10.0 sec  7.35 GBytes  6.31 Gbits/sec
[  6] Sent 891785 datagrams
[  5]  0.0-10.0 sec  7.73 GBytes  6.64 Gbits/sec
[  5] Sent 937903 datagrams
[  3]  0.0-10.0 sec  1.48 GBytes  1.28 Gbits/sec
[  3] Sent 180175 datagrams
[  4]  0.0-10.0 sec  7.79 GBytes  6.69 Gbits/sec
[  4] Sent 945661 datagrams
[SUM]  0.0-10.0 sec  24.4 GBytes  20.9 Gbits/sec
[SUM] Sent 2955524 datagr

In [5]:
h1 = slice.get_node(name=h1_name) 
stdout, stderr = h1.execute('tmux new -d \'iperf -s -f K\'')
print (stdout)
print (stderr)

In [6]:
#TCP TEST:
#Backward direction
h2 = slice.get_node(name=h2_name) 
stdout, stderr = h2.execute(f'iperf -c 172.16.0.1 -P 32 -w 999M') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.0.1, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 33] local 172.16.0.2 port 45928 connected with 172.16.0.1 port 5001
[ 30] local 172.16.0.2 port 45926 connected with 172.16.0.1 port 5001
[ 34] local 172.16.0.2 port 45932 connected with 172.16.0.1 port 5001
[ 31] local 172.16.0.2 port 45930 connected with 172.16.0.1 port 5001
[  5] local 172.16.0.2 port 45872 connected with 172.16.0.1 port 5001
[  3] local 172.16.0.2 port 45876 connected with 172.16.0.1 port 5001
[  7] local 172.16.0.2 port 45878 connected with 172.16.0.1 port 5001
[  4] local 172.16.0.2 port 45870 connected with 172.16.0.1 port 5001
[  6] local 172.16.0.2 port 45874 connected with 172.16.0.1 port 5001
[ 10] local 172.16.0.2 port 45880 connected with 172.16.0.1 port 5001
[  8] local 172.16.0.2 port 45882 connected with 172.16.0.1 port 5001
[

In [7]:
#UDP TEST:
#Backward direction
h2 = slice.get_node(name=h2_name) 
stdout, stderr = h2.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.0.1 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.0.1, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  6] local 172.16.0.2 port 35770 connected with 172.16.0.1 port 5001
[  3] local 172.16.0.2 port 53067 connected with 172.16.0.1 port 5001
[  4] local 172.16.0.2 port 41318 connected with 172.16.0.1 port 5001
[  5] local 172.16.0.2 port 52709 connected with 172.16.0.1 port 5001
[ ID] Interval       Transfer     Bandwidth
[  6]  0.0-10.0 sec  9.67 GBytes  8.31 Gbits/sec
[  6] Sent 1173662 datagrams
[  5]  0.0-10.0 sec  9.08 GBytes  7.80 Gbits/sec
[  5] Sent 1101427 datagrams
[  3]  0.0-10.0 sec  9.27 GBytes  7.97 Gbits/sec
[  3] Sent 1125388 datagrams
[  4]  0.0-10.0 sec  1.28 GBytes  1.10 Gbits/sec
[  4] Sent 155189 datagrams
[SUM]  0.0-10.0 sec  29.3 GBytes  25.2 Gbits/sec
[SUM] Sent 3555666 dat

In [8]:
h5 = slice.get_node(name=h5_name) 
stdout, stderr = h5.execute('tmux new -d \'iperf -s -f K\'')
print (stdout)
print (stderr)

In [9]:
#Backward direction:
h4 = slice.get_node(name=h4_name) 
stdout, stderr = h4.execute(f'iperf -c 172.16.2.2 -P 32 -w 999M -t 30') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.2.2, TCP port 5001
TCP window size: 1.00 GByte (WARNING: requested  999 MByte)
------------------------------------------------------------
[ 34] local 172.16.2.1 port 54208 connected with 172.16.2.2 port 5001
[ 31] local 172.16.2.1 port 54202 connected with 172.16.2.2 port 5001
[ 33] local 172.16.2.1 port 54204 connected with 172.16.2.2 port 5001
[ 35] local 172.16.2.1 port 54206 connected with 172.16.2.2 port 5001
[  3] local 172.16.2.1 port 54148 connected with 172.16.2.2 port 5001
[  4] local 172.16.2.1 port 54146 connected with 172.16.2.2 port 5001
[  5] local 172.16.2.1 port 54150 connected with 172.16.2.2 port 5001
[  6] local 172.16.2.1 port 54152 connected with 172.16.2.2 port 5001
[  7] local 172.16.2.1 port 54154 connected with 172.16.2.2 port 5001
[  8] local 172.16.2.1 port 54156 connected with 172.16.2.2 port 5001
[ 10] local 172.16.2.1 port 54160 connected with 172.16.2.2 port 5001
[

In [10]:
#Forward direction
h4 = slice.get_node(name=h4_name) 
stdout, stderr = h4.execute(f'iperf -l8848 -T30 -u -w4m -b10G -c 172.16.2.2 -P 4') 
print (stdout)
print (stderr)

------------------------------------------------------------
Client connecting to 172.16.2.2, UDP port 5001
Sending 8848 byte datagrams, IPG target: 6.59 us (kalman adjust)
UDP buffer size: 7.63 MByte (WARNING: requested 3.81 MByte)
------------------------------------------------------------
[  6] local 172.16.2.1 port 40480 connected with 172.16.2.2 port 5001
[  3] local 172.16.2.1 port 35060 connected with 172.16.2.2 port 5001
[  4] local 172.16.2.1 port 57478 connected with 172.16.2.2 port 5001
[  5] local 172.16.2.1 port 39793 connected with 172.16.2.2 port 5001
[ ID] Interval       Transfer     Bandwidth
[  6]  0.0-10.0 sec  9.08 GBytes  7.80 Gbits/sec
[  6] Sent 1101623 datagrams
[  3]  0.0-10.0 sec  9.09 GBytes  7.80 Gbits/sec
[  3] Sent 1102504 datagrams
[  5]  0.0-10.0 sec  5.50 GBytes  4.72 Gbits/sec
[  5] Sent 667385 datagrams
[  4]  0.0-10.0 sec  6.25 GBytes  5.37 Gbits/sec
[  4] Sent 758374 datagrams
[SUM]  0.0-10.0 sec  29.9 GBytes  25.7 Gbits/sec
[SUM] Sent 3629886 data

# Upload File and Execute

In [22]:
#upload_file(local_file_path, remote_file_path, retry=3, retry_interval=10)
# FILE 1: scistream source code

try:
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('../scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('../scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('../scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('../scistream-source-code.zip',"scistream-source-code.zip", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [23]:
#FILE 2: setup.sh
try:
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('../setup.sh',"setup.sh", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('../setup.sh',"setup.sh", retry=3, retry_interval=10)
   
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('../setup.sh',"setup.sh", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('../setup.sh',"setup.sh", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [24]:
#Optional file: QUIC proxy exmaple
try:
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('../quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('../quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
   
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('../quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('../quic-tun_0.0.1_linux_amd64.tar.gz',"quic-tun_0.0.1_linux_amd64.tar.gz", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [25]:
#Optional file: QUIC tunnel example
try:
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('../quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('../quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    h3 = slice.get_node(name=h3_name)
    h3.upload_file('../quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('../quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('../quic-proxy-example.zip',"quic-proxy-example.zip", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

Error: Node not found: h3


Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 739, in get_node
    return Node.get_node(self, self.get_fim_topology().nodes[name])
  File "/opt/conda/lib/python3.9/site-packages/fim/view_only_dict.py", line 19, in __getitem__
    return self.__dict.__getitem__(k)
KeyError: 'h3'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_646/2642736187.py", line 8, in <module>
    h3 = slice.get_node(name=h3_name)
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 742, in get_node
    raise Exception(f"Node not found: {name}")
Exception: Node not found: h3


In [26]:
#Optional file: quic distribuation - quic-go version...
try:
    slice = fablib.get_slice(name=slice_name)
    h1 = slice.get_node(name=h1_name)
    h1.upload_file('../quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    
    h2 = slice.get_node(name=h2_name)
    h2.upload_file('../quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    h3 = slice.get_node(name=h3_name)
    h3.upload_file('../quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    h4 = slice.get_node(name=h4_name)
    h4.upload_file('../quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    h5 = slice.get_node(name=h5_name)
    h5.upload_file('../quic-go.zip',"quic-go.zip", retry=3, retry_interval=10)
    
    #stdout, stderr = h3.execute(host_config_script_h3)
    #print("stdout: {}".format(stdout))
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

Error: Node not found: h3


Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 739, in get_node
    return Node.get_node(self, self.get_fim_topology().nodes[name])
  File "/opt/conda/lib/python3.9/site-packages/fim/view_only_dict.py", line 19, in __getitem__
    return self.__dict.__getitem__(k)
KeyError: 'h3'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_646/1409364764.py", line 9, in <module>
    h3 = slice.get_node(name=h3_name)
  File "/opt/conda/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 742, in get_node
    raise Exception(f"Node not found: {name}")
Exception: Node not found: h3


In [27]:
#FILE 3: mona demo, only upload on h1 and h5
# try:
#     slice = fablib.get_slice(name=slice_name)
#     h1 = slice.get_node(name=h1_name)
#     h1.upload_file('mona-demo.zip',"mona-demo.zip", retry=3, retry_interval=10)
    
#     h5 = slice.get_node(name=h5_name)
#     h5.upload_file('mona-demo.zip',"mona-demo.zip", retry=3, retry_interval=10)
    
#     #stdout, stderr = h3.execute(host_config_script_h3)
#     #print("stdout: {}".format(stdout))
# except Exception as e:
#     print(f"Error: {e}")
#     traceback.print_exc()

In [28]:
#Optional file: tls certs for public usage. Ignore if test-only 
# try:
#     slice = fablib.get_slice(name=slice_name)
#     h1 = slice.get_node(name=h1_name)
#     h1.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
    
#     h2 = slice.get_node(name=h2_name)
#     h2.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
#     h3 = slice.get_node(name=h3_name)
#     h3.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
#     h4 = slice.get_node(name=h4_name)
#     h4.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
#     h5 = slice.get_node(name=h5_name)
#     h5.upload_directory('newcerts/aioquic/newcerts',"newcerts", retry=3, retry_interval=10)
    
#     #stdout, stderr = h3.execute(host_config_script_h3)
#     #print("stdout: {}".format(stdout))
# except Exception as e:
#     print(f"Error: {e}")
#     traceback.print_exc()

In [29]:
#Optional download file: dumpfile...
# try:
#     slice = fablib.get_slice(name=slice_name)
#     h1 = slice.get_node(name=h1_name)
#     h1.download_file("dumpFile/prodGn_LAN.cap",'prodGn_LAN.cap', retry=3, retry_interval=10)
#     h2 = slice.get_node(name=h2_name)
#     h2.download_file("dumpFile/prodGn_WAN.cap",'prodGn_WAN.cap', retry=3, retry_interval=10)
#     h4 = slice.get_node(name=h4_name)
#     h4.download_file("dumpFile/consGn_WAN.cap",'consGn_WAN.cap', retry=3, retry_interval=10)
#     h5 = slice.get_node(name=h5_name)
#     h5.download_file("dumpFile/consGn_LAN.cap",'consGn_LAN.cap', retry=3, retry_interval=10)
#     h5 = slice.get_node(name=h5_name)
#     h5.download_file("dumpFile/ssl_log.txt",'aioquic/ssl_log.txt', retry=3, retry_interval=10)
# except Exception as e:
#     print(f"Error: {e}")
#     traceback.print_exc()

## Configure Nodes ----VERBOSE


In [30]:
master_script = "bash setup.sh"
try:
    h1 = slice.get_node(name=h1_name)        
    stdout, stderr = h1.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h1...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h2 = slice.get_node(name=h2_name)
    stdout, stderr = h2.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h2...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h4 = slice.get_node(name=h4_name) 
    stdout, stderr = h4.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h4...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    
try:
    h5 = slice.get_node(name=h5_name) 
    stdout, stderr = h5.execute(master_script)
    #print("stdout: {}".format(stdout))
    print("done h5...")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

done h1...
done h2...
done h4...
done h5...


## Below are some optinal operations for the slice

### Extend the slice reservation time

In [31]:
# import datetime

# #Set end host to now plus 1 day
# end_date = (datetime.datetime.utcnow() + datetime.timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S")

# try:
#     slice = fablib.get_slice(name=slice_name)

#     slice.renew(end_date)
# except Exception as e:
#     print(f"Exception: {e}")

### Check New Lease End Date

In [32]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     print(f"Lease End         : {slice.get_lease_end()}")
       
# except Exception as e:
#     print(f"Exception: {e}")

### DANGER ZONE --------*-------- Delete Slice

In [33]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Fail: {e}")
#     traceback.print_exc()